In [1]:
import os

from torch.cuda import OutOfMemoryError

os.environ['USER_DIR'] = 'unknown'

import json
from pathlib import Path
from nfmc.util import create_flow_object
import torch
import time
from slurm.constants import SUPPORTED_NORMALIZING_FLOWS
import numpy as np

In [2]:
import itertools
from tqdm import tqdm

dimensionality_list = [2, 4, 8, 16, 32, 64, 128]
n_chains = 100
n_measurements = 20

results_dir = Path('./cpu-gpu-results')
results_dir.mkdir(exist_ok=True)

configs = itertools.product(['cuda', 'cpu'], SUPPORTED_NORMALIZING_FLOWS, dimensionality_list)


for device, flow, dimensionality in (pbar := tqdm(configs)):
    results_file = results_dir / f'{flow}-{dimensionality}-{device}.json'
    pbar.set_description(f'{device}-{flow}-{dimensionality}')
    if results_file.exists():
        continue

    torch.manual_seed(0)
    event_shape = (dimensionality,)

    torch.manual_seed(0)
    x = torch.randn(size=(n_chains, *event_shape)).to(device)

    log_prob_times = []
    standard_backward_times = []
    sample_times = []
    sample_backward_times = []
    
    total_elapsed_time = 0  # seconds
    for _ in range(n_measurements):
        try:
            flow_obj = create_flow_object(flow, event_shape).to(device)
        except OutOfMemoryError:
            continue

        # Measure log_prob time once
        try:
            t0 = time.time()
            standard_log_prob = flow_obj.log_prob(x)
            log_prob_times.append(time.time() - t0)

            t0 = time.time()
            loss = -standard_log_prob.mean()
            loss.backward()
            standard_backward_times.append(time.time() - t0)
        except (NotImplementedError, ValueError) as e:
            log_prob_times.append(np.nan)
            standard_backward_times.append(np.nan)
        except OutOfMemoryError:
            log_prob_times.append(np.nan)
            standard_backward_times.append(np.nan)

        flow_obj.zero_grad()

        # Measure sample time once
        try:
            t0 = time.time()
            x_sampled, log_prob_sampled = flow_obj.sample((n_chains,), return_log_prob=True)
            sample_times.append(time.time() - t0)

            t0 = time.time()
            loss = -log_prob_sampled.mean()
            loss.backward()
            sample_backward_times.append(time.time() - t0)
        except (NotImplementedError, ValueError) as e:
            sample_times.append(np.nan)
            sample_backward_times.append(np.nan)
        except OutOfMemoryError:
            sample_times.append(np.nan)
            sample_backward_times.append(np.nan)

        flow_obj.zero_grad()

    with open(results_file, 'w') as f:
        json.dump({
            'log_prob_average_time_seconds': sum(log_prob_times) / len(log_prob_times),
            'standard_backward_average_time_seconds': sum(standard_backward_times) / len(
                standard_backward_times),
            'sample_average_time_seconds': sum(sample_times) / len(sample_times),
            'sample_backward_average_time_seconds': sum(sample_backward_times) / len(sample_backward_times),
            'n_dim': dimensionality,
            'device': device,
            'flow': flow
        }, f)

cpu-ia-naf-deep-dense-64: : 264it [6:46:13, 92.33s/it]  

KeyboardInterrupt



In [4]:
import itertools
from tqdm import tqdm

results = []

dimensionality_list = [2, 4, 8, 16, 32, 64, 128]
n_chains = 100
n_measurements = 20

configs = itertools.product(['cuda', 'cpu'], SUPPORTED_NORMALIZING_FLOWS, dimensionality_list)

for m in range(n_measurements):
    for device, flow, dimensionality in (pbar := tqdm(configs)):
        pbar.set_description(f'{device}-{flow}-{dimensionality}')

        torch.manual_seed(0)
        event_shape = (dimensionality,)

        torch.manual_seed(0)
        x = torch.randn(size=(n_chains, *event_shape)).to(device)

        log_prob_times = []
        standard_backward_times = []
        sample_times = []
        sample_backward_times = []

        total_elapsed_time = 0  # seconds
        try:
            flow_obj = create_flow_object(flow, event_shape).to(device)
        except OutOfMemoryError:
            continue

        # Measure log_prob time once
        try:
            t0 = time.time()
            standard_log_prob = flow_obj.log_prob(x)
            log_prob_times.append(time.time() - t0)

            t0 = time.time()
            loss = -standard_log_prob.mean()
            loss.backward()
            standard_backward_times.append(time.time() - t0)
        except (NotImplementedError, ValueError) as e:
            log_prob_times.append(np.nan)
            standard_backward_times.append(np.nan)
        except OutOfMemoryError:
            log_prob_times.append(np.nan)
            standard_backward_times.append(np.nan)

        flow_obj.zero_grad()

        # Measure sample time once
        try:
            t0 = time.time()
            x_sampled, log_prob_sampled = flow_obj.sample((n_chains,), return_log_prob=True)
            sample_times.append(time.time() - t0)

            t0 = time.time()
            loss = -log_prob_sampled.mean()
            loss.backward()
            sample_backward_times.append(time.time() - t0)
        except (NotImplementedError, ValueError) as e:
            sample_times.append(np.nan)
            sample_backward_times.append(np.nan)
        except OutOfMemoryError:
            sample_times.append(np.nan)
            sample_backward_times.append(np.nan)

            flow_obj.zero_grad()

        results.append({
            'flow': flow,
            'device': device,
            'dimensionality': dimensionality,
            'log_prob_time': log_prob_times[0],
            'standard_backward_time': standard_backward_times[0],
            'sample_time': sample_times[0],
            'sample_backward_time': sample_backward_times[0],
        })

        with open('cpu-gpu-results.json', 'w') as f:
            json.dump(results, f)

cpu-sylvester-128: : 378it [5:53:16, 56.08s/it]          
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
